In [2]:
import pandas as pd
import dask.dataframe as dd
import nltk
import numpy as np
import swifter
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pedromadruga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
%%time

dataset_name = "jobindex_cropped_bigger"

import os.path
extension = os.path.basename(f'../data/interim/{dataset_name}.csv')

print(extension)

df = pd.read_csv(f'../data/interim/{dataset_name}.csv')

jobindex_cropped_bigger.csv
CPU times: user 113 ms, sys: 17.1 ms, total: 130 ms
Wall time: 147 ms


In [4]:
df_init = df[['title', 'company', 'description','location','link','ratings_link', 'source', 'date']]

In [5]:
%%time

df_init['merged'] = (
                        df_init['title'].fillna('') + ' '
                        + df_init['company'].fillna('') + ' '
                        + df_init['description'].fillna('')+ ' '
                        + df_init['location'].fillna('')+ ' '
                        + df_init['link'].astype(str).fillna('')+ ' '
                        + df_init['ratings_link'].fillna('')+ ' '
                        + df_init['source'].fillna('')+ ' '
                        + df_init['description'].fillna('')+ ' '
                        + df_init['description'].fillna('')+ ' '
                        + df_init['date'].astype(str).fillna('')
                        )

df_init['merged'] = df_init['merged'].astype(str)

CPU times: user 97.8 ms, sys: 18.5 ms, total: 116 ms
Wall time: 127 ms


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
sample = df_init['merged'][:1000]

print(sample.to_list())

['Østervang søger en social- og sundhedshjælper med interesse og engagement inden for demensområdet pr. 1. maj 2020 eller snarest mulig til en aftenvagtstilling Ikast-Brande Kommune Ikast-Brande Kommune, KlovborgStillingen er på 30 timer eller efter individuelt ønske om andet timeantal. Der arbejdes fortrinsvis i aftenvagt, men der kan være enkelte dagvagter eller nattevagter. Østervang er et plejecenter med plads til 28 beboere fordelt på 2 afdelinger med 14 boliger på hver. Den ene afdeling er for beboere, som har en demenssygdom og derfor har behov for de demensvenlige rammer, som afdelingen er indrettet med. Vi tilbyder dig: En spændende arbejdsplads med engagerede kollegaerEn arbejdsplads hvor vi i fællesskab skaber gode løsninger i hverdagen og samarbejder om det gode arbejdsmiljøEn arbejdsplads hvor indflydelse, medbestemmelse og medansvar er en selvfølgelighed Klovborg http://www.ikast-brande.dk/ /virksomhed/26512/ikast-brande-kommune#show_rating  Ikast-Brande Kommune, Klovborg

In [7]:
%%time

import re
import numpy as np
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.stem.snowball import DanishStemmer
da_stop_words = stopwords.words('danish')
import lemmy

lemmatizer = lemmy.load("da")
stemmer = DanishStemmer()

all_descriptions = np.array(df_init['merged'])
all_descriptions_wo_stopwords = []

# def preprocess_text(merged_features):
#     all_descriptions_wo_stopwords = []
        
#     ## remove stopwords
#     for sentence in merged_features:
#         removed_stop_words = [x for x in sentence.split() if not x in da_stop_words]
#         result = ' '.join(removed_stop_words)
#         all_descriptions_wo_stopwords.append(result)
    
#     return all_descriptions_wo_stopwords


def pp_text(text):
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    
    textblob = TextBlob(text)
#     print('\n initial length: ', len(textblob.words))
    singles = [stemmer.stem(word) for word in textblob.words]
#     print('\n singles length: ', len(singles))
    no_stop_words = [word for word in singles if not word in da_stop_words]
#     print('\n no stop words length: ', len(no_stop_words))
    joined_text = " ".join(no_stop_words)
    final_text = lemmatizer.lemmatize("", joined_text)
#     print('\n final text length: ', len(final_text), '\n\n\n')
    
    return final_text[0]

df_init['merged'][0]

CPU times: user 305 ms, sys: 31.1 ms, total: 336 ms
Wall time: 427 ms


'Østervang søger en social- og sundhedshjælper med interesse og engagement inden for demensområdet pr. 1. maj 2020 eller snarest mulig til en aftenvagtstilling Ikast-Brande Kommune Ikast-Brande Kommune, KlovborgStillingen er på 30 timer eller efter individuelt ønske om andet timeantal. Der arbejdes fortrinsvis i aftenvagt, men der kan være enkelte dagvagter eller nattevagter. Østervang er et plejecenter med plads til 28 beboere fordelt på 2 afdelinger med 14 boliger på hver. Den ene afdeling er for beboere, som har en demenssygdom og derfor har behov for de demensvenlige rammer, som afdelingen er indrettet med. Vi tilbyder dig: En spændende arbejdsplads med engagerede kollegaerEn arbejdsplads hvor vi i fællesskab skaber gode løsninger i hverdagen og samarbejder om det gode arbejdsmiljøEn arbejdsplads hvor indflydelse, medbestemmelse og medansvar er en selvfølgelighed Klovborg http://www.ikast-brande.dk/ /virksomhed/26512/ikast-brande-kommune#show_rating  Ikast-Brande Kommune, KlovborgS

In [8]:
%time df_init['merged_2'] = df_init['merged'].swifter.apply(pp_text)

df_init.to_csv(f'../data/processed/{dataset_name}.csv')

df_init['merged_2'][0]


CPU times: user 1min 13s, sys: 1.52 s, total: 1min 14s
Wall time: 1min 18s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'østervang søg social sundhedshjælp interes engagement demensområd pr 1 maj 2020 ell snarest mul aftenvagtstilling ikastbrand kommun ikastbrand kommun klovborgstilling 30 tim ell eft individuelt ønsk and timeantal arbejd fortrinsvis aftenvag kan vær enkelt dagvag ell nattevag østervang plejecent plad 28 bebo fordelt 2 afdeling 14 bol hver ene afdeling bebo demenssygdom derfor behov demensven ram afdeling indret tilbyd spænd arbejdsplad engag kollega arbejdsplad fællesskab skab god løsning hverdag samarbejd god arbejdsmiljø arbejdsplad indflyd medbestem medansvar selvfølg klovborg httpwwwikastbrandedk virksomhed26512ikastbrandekommuneshow_rating ikastbrand kommun klovborgstilling 30 tim ell eft individuelt ønsk and timeantal arbejd fortrinsvis aftenvag kan vær enkelt dagvag ell nattevag østervang plejecent plad 28 bebo fordelt 2 afdeling 14 bol hver ene afdeling bebo demenssygdom derfor behov demensven ram afdeling indret tilbyd spænd arbejdsplad engag kollega arbejdsplad fællesskab ska

In [9]:
df_init['merged_2'][0]

'østervang søg social sundhedshjælp interes engagement demensområd pr 1 maj 2020 ell snarest mul aftenvagtstilling ikastbrand kommun ikastbrand kommun klovborgstilling 30 tim ell eft individuelt ønsk and timeantal arbejd fortrinsvis aftenvag kan vær enkelt dagvag ell nattevag østervang plejecent plad 28 bebo fordelt 2 afdeling 14 bol hver ene afdeling bebo demenssygdom derfor behov demensven ram afdeling indret tilbyd spænd arbejdsplad engag kollega arbejdsplad fællesskab skab god løsning hverdag samarbejd god arbejdsmiljø arbejdsplad indflyd medbestem medansvar selvfølg klovborg httpwwwikastbrandedk virksomhed26512ikastbrandekommuneshow_rating ikastbrand kommun klovborgstilling 30 tim ell eft individuelt ønsk and timeantal arbejd fortrinsvis aftenvag kan vær enkelt dagvag ell nattevag østervang plejecent plad 28 bebo fordelt 2 afdeling 14 bol hver ene afdeling bebo demenssygdom derfor behov demensven ram afdeling indret tilbyd spænd arbejdsplad engag kollega arbejdsplad fællesskab ska

In [10]:
%%time
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(df_init['merged'])

X.toarray().shape

CPU times: user 2.56 s, sys: 695 ms, total: 3.26 s
Wall time: 3.74 s


(10000, 45986)

In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity, pairwise_kernels
# from tensorflow import keras, SparseTensor, transpose
# import tensorflow as tf

# def tf_cosine_similarity(u, v):
#     %time u = keras.backend.expand_dims(u, 1)

#     %time n = tf.reduce_sum(u * v, axis=2)
#     %time d = tf.linalg.norm(u, axis=2) * tf.linalg.norm(v, axis=1)

#     return n / d


# def convert_sparse_matrix_to_sparse_tensor(X):
#     coo = X.tocoo()
#     indices = np.mat([coo.row, coo.col]).transpose()
    
#     return SparseTensor(indices, coo.data, coo.shape)


# def compute_cosine_distances(u,v):
#     u = tf.expand_dims(u, 1)

#     n = tf.reduce_sum(u * v, axis=2)
#     d = tf.linalg.norm(u, axis=2) * tf.linalg.norm(v, axis=1)
    
#     return n/d
        
# print(convert_sparse_matrix_to_sparse_tensor(X).numpy())
    
# compute_cosine_distances(X.todense(),X.todense())

# %time tf_cosine_similarity(X.toarray(), X.toarray())

# df_matrix = pd.DataFrame(data=similarity_matrix2)

# file_name = "../data/interim/similarity_matrix-" + round(time.time()) + ".csv"

# df_matrix.to_csv(file_name)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity, pairwise_kernels
from tqdm.notebook import tqdm

def search(job_title):
    direct_search_results = [df_init.index[df_init['merged'].str.contains(word, case=False)].values for word in job_title.split()]
    return list(set(direct_search_results[0]).intersection(*direct_search_results))

search_results = search("AWS architect")

print('direct search results id list: ', search_results)
print("\n\n\n\n")
first_result = search_results[0]

# first_result



# print(search_results[1])
# print(df['title'][first_result])

# for index, value in enumerate()

# distances = pairwise_kernels(X, X[first_result], n_jobs=-1, metric="cosine")

nherp = []

for index,result in enumerate(search_results):
    distances = pairwise_kernels(X, X[result], n_jobs=-1, metric='cosine')
    # print(distance)
    
    flat_distances = np.array(distances).flatten()
    
    for index, distance in enumerate(flat_distances):
        nherp.append({
        'distance': float(distance),
        'index': index,
        'result_title': df['title'][result],
        })
    

# print(len(search_results))
# all_distances = np.array(nherp).flatten()
# print('nherp is: ', len(np.array(nherp).flatten()))


# for index, value in enumerate(stuff):
# #     print(value, index)
#     break


    
# sorted_similarities = sorted(range(len(distances)), key=lambda k: distances[k], reverse=True)[:10]

sorted_similarities = sorted(nherp, key=lambda k: k['distance'], reverse=True)[:50]

for index,value in tqdm(enumerate(sorted_similarities)):
    print(value)
    
    
# mylist = [3,6,3,2,4,8,23]
# sorted(mylist)

# sorted

# df_filtered['title'][5122]

direct search results id list:  [9323, 9324]







{'distance': 1.0000000000000002, 'index': 9323, 'result_title': 'Azure/AWS Cloud Architect'}
{'distance': 1.0000000000000002, 'index': 9324, 'result_title': 'Azure and AWS architect, developer and DevOps'}
{'distance': 0.5816600588826344, 'index': 8588, 'result_title': 'Azure and AWS architect, developer and DevOps'}
{'distance': 0.5798571397619933, 'index': 8587, 'result_title': 'Azure and AWS architect, developer and DevOps'}
{'distance': 0.5585102900279626, 'index': 8586, 'result_title': 'Azure and AWS architect, developer and DevOps'}
{'distance': 0.339984501010406, 'index': 8597, 'result_title': 'Azure and AWS architect, developer and DevOps'}
{'distance': 0.3399191647018317, 'index': 3093, 'result_title': 'Azure and AWS architect, developer and DevOps'}
{'distance': 0.33872135778424656, 'index': 4783, 'result_title': 'Azure/AWS Cloud Architect'}
{'distance': 0.3188774095644255, 'index': 5167, 'result_title': 'Azure/AWS Cloud Architect'}
{'distance': 0.3048931320442363, 'index': 8

In [ ]:

print(distances[30658])

df_filtered['title'].compute()[30658]

In [ ]:
rr = [{'distance': 0.10066021580103635, 'index': 0, 'result_title': 'PenSam Bank søger kunderådgiver'}, {'distance': 0.9, 'index': 0, 'result_title': 'another'}]


sorted(rr, key=lambda k: k['distance'], reverse=True)

In [ ]:
from tqdm.notebook import tqdm
from functools import reduce

similarities = []    

def search(job_title):
        
    stuff = [df_merged.index[df_merged['merged'].str.contains(word, case=False)].values for word in job_title.split()]
    return list(set(stuff[0]).intersection(*stuff))

def cosine_similarity(job_title):    
    search_matching_indeces = search(job_title)
    
    
    for matching_index in search_matching_indeces:
        for idx,similarity_value in enumerate(tqdm(similarity_matrix[matching_index])):
            
            similarities.append({
                "title": df_merged['title'][idx],
                "similarity": similarity_value,
                "description": df_init['description'][idx],
                "location": df_init['location'][idx],
                "exact match": True if matching_index == idx else False
            })
        

    sorted_similarities = sorted(similarities, key=lambda x: x['similarity'], reverse=True)
    
    ## just return the top X similarities
    return sorted_similarities[:100]
    
cosine_similarity('sygeplejerske psykiatriske')


In [ ]:
%%time
convert_sparse_matrix_to_sparse_tensor(X)

print(X)
print(X.toarray())


# loss = tf.keras.losses.cosine_similarity(X.toarray(),X.toarray(), axis=1)
# print((-1*loss).numpy())

In [ ]:
%%time
import tqdm
from sklearn.metrics.pairwise import cosine_similarity




In [16]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

# Corpus with example sentences
corpus = df_merged['merged'][2]
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['Risk manager']


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))

NameError: name 'df_merged' is not defined

In [1]:
'hello'.split(',')[0] + '_tfidf.csv'

'hello_tfidf.csv'